In [1]:
import os
import csv
from config import database, datadir
from sqlalchemy.orm import sessionmaker
from database.tools import init_engine
from database.models import (Base, Participant, Case, Control, Can, Dors, Sv, Ov)
import pandas as pd
import numpy as np

## Open session

In [2]:
engine = init_engine(database["username"],
                     database["password"],
                     database["dbname"])

# Delete all children tables
tables = [
    Control, 
    Case, 
    Dors, 
    Can, 
    Ov, 
    Sv
]

for table in tables:
    if engine.has_table(table.__tablename__):
        table.__table__.drop(bind=engine)

# Finally, the parent table
if engine.has_table("participant"):
    Participant.__table__.drop(bind=engine)

Base.metadata.create_all(engine)
Base.prepare(engine)
Session = sessionmaker(engine)
session = Session()
session.close()

## Upload unique lopnrs

In [12]:
# We need a table with all unique lop numbers to use relationships in SQL.
# This will be the participant master table, collecting all individuals. We
# will call it "participant". The table will serve as a hub connecting
# all tables together.



# Create empty participants df
participants = pd.DataFrame({"lopnr": []})

# Add cases
cases = os.path.join(datadir, "cases.csv")
participants = extract_unique_columns(cases, "Lopnr_fall", participants, "lopnr")

# Add controls
controls = os.path.join(datadir, "controls.csv")
participants = extract_unique_columns(controls, "Lopnr_kontroll", participants, "lopnr")

# Add can
can = os.path.join(datadir, "can.csv")
participants = extract_unique_columns(can, "LopNr", participants, "lopnr", nrows=100000)

# Add dors
dors = os.path.join(datadir, "dors.csv")
participants = extract_unique_columns(dors, "LopNr", participants, "lopnr", nrows=100000)

# Add ov
ov = os.path.join(datadir, "ov.csv")
participants = extract_unique_columns(ov, "LopNr", participants, "lopnr", nrows=100000)

# Add ov
sv = os.path.join(datadir, "sv.csv")
participants = extract_unique_columns(sv, "LopNr", participants, "lopnr", nrows=100000)

In [14]:
participants.to_sql("participant", engine, if_exists="append", chunksize=2**8, index=False)

## Load the rest of the data

In [6]:
# Load csvs
cases = pd.read_csv(os.path.join(datadir, "cases.csv"))
controls = pd.read_csv(os.path.join(datadir, "controls.csv"))
ov = pd.read_csv(os.path.join(datadir, "ov.csv"), nrows=100000)
sv = pd.read_csv(os.path.join(datadir, "sv.csv"), nrows=100000)
can = pd.read_csv(os.path.join(datadir, "can.csv"), nrows=100000)
dors = pd.read_csv(os.path.join(datadir, "dors.csv"), nrows=100000, encoding="latin-1")

# Change column names to fit with database models in sqlalchemy
cases.columns = ["participant_lopnr"] + [val.lower() for val in cases.columns[1:]]
cases.columns[0] = "participant_lopnr"
controls.columns = ["participant_lopnr", "case_lopnr"] + [val.lower() for val in controls.columns[2:]]
sv.columns = ["participant_lopnr"] + [val.lower() for val in sv.columns[1:]]
ov.columns = ["participant_lopnr"] + [val.lower() for val in ov.columns[1:]]
dors.columns = ["participant_lopnr"] + [val.lower() for val in dors.columns[1:]]
can.columns = ["participant_lopnr"] + [val.lower() for val in can.columns[1:]]

/home/mab/.conda/envs/istopmm_py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mab/.conda/envs/istopmm_py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


TypeError: Index does not support mutable operations

In [8]:
cases = os.path.join(datadir, "cases.csv")
df_cases = pd.read_csv(cases)
cols = list(df_cases.columns)
cols[0] = "participant_lopnr"
cols[1] = "case_lopnr"
df_cases.columns = cols

## Everything to MySQL db

In [4]:

# Weird non-numeric entries need to be NULL.
sv.sjukhus = sv.sjukhus.apply(int_or_null).astype(float)
ov.sjukhus = ov.sjukhus.apply(int_or_null).astype(float)
can.klinik = can.klinik.apply(int_or_null)

In [ ]:
cases.to_sql("case", engine, if_exists="append", chunksize=20000, index=False)
controls.to_sql("control", engine, if_exists="append", chunksize=20000, index=False)

In [5]:
sv.to_sql("sv", engine, if_exists="append", chunksize=2**8, index=False)
ov.to_sql("ov", engine, if_exists="append", chunksize=2**8, index=False)
can.to_sql("can", engine, if_exists="append", chunksize=2**8, index=False)
dors.to_sql("dors", engine, if_exists="append", chunksize=2**8, index=False)